<a href="https://colab.research.google.com/github/LilliLee-1318/UIUC-helper-chatbot/blob/main/Set_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install -q transformers accelerate bitsandbytes sentence-transformers faiss-cpu gradio

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import gradio as gr


model_id = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16
)

print("✅ Mistral-7B Instruct loaded successfully on A100!")


docs = [
    "UIUC Counseling Center is located at Turner Hall, Room 45.",
    "CS 124 uses Kotlin as its main programming language.",
    "Dining halls at UIUC are open from 7 AM to 8 PM.",
    "McKinley Health Center provides medical services for students.",
    "The UIUC Main Library has over 14 million volumes."
]

embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeds = embedder.encode(docs)
index = faiss.IndexFlatL2(doc_embeds.shape[1])
index.add(np.array(doc_embeds))
print("✅ FAISS vector index ready!")


def retrieve(query, k=2):
    query_vec = embedder.encode([query])
    _, idxs = index.search(np.array(query_vec), k)
    return [docs[i] for i in idxs[0]]

def chat_with_bot(user_input):
    retrieved = retrieve(user_input, k=2)
    context = "\n".join(retrieved)

    prompt = f"""You are a friendly and knowledgeable assistant for University of Illinois Urbana-Champaign (UIUC) students.
Answer the question accurately and concisely based on the context below.

Context:
{context}

Question:
{user_input}
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=180)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip()


uiuc_blue = "#13294B"
uiuc_orange = "#FF552E"

with gr.Blocks(css=f"""
body {{
    background-color: #f8f9fb;
    font-family: 'Inter', sans-serif;
}}
.gradio-container {{
    border-radius: 20px;
    box-shadow: 0 0 15px rgba(0,0,0,0.1);
}}
h1, h2, h3 {{
    color: {uiuc_blue};
    text-align: center;
}}
button {{
    background-color: {uiuc_orange} !important;
    color: white !important;
    border-radius: 10px !important;
    font-weight: 600 !important;
}}
textarea {{
    border: 2px solid {uiuc_blue} !important;
    border-radius: 10px !important;
}}
""") as demo:
    gr.Markdown(f"""
    <div style='text-align:center;'>
        <h1 style='color:{uiuc_blue}; font-size:30px; font-weight:800;'>
            🎓 UIUC Helper Chatbot
        </h1>
        <p style='color:{uiuc_orange}; font-size:16px; font-weight:500;'>
            Powered by Mistral-7B-Instruct | University of Illinois Urbana-Champaign
        </p>
    </div>
    """)

    chatbot = gr.Chatbot(
        label="Chat with UIUC Helper 🤖",
        bubble_full_width=False,
        show_copy_button=True,
        avatar_images=["https://upload.wikimedia.org/wikipedia/en/0/09/Illinois_Fighting_Illini_logo.svg", None]
    )

    msg = gr.Textbox(
        label="Ask me anything about UIUC!",
        placeholder="e.g., Where is the Counseling Center located?",
        lines=2
    )

    send = gr.Button("Send ✉️")

    def greet():
        return [("👋 Hi there! I'm your UIUC Helper Bot. Ask me anything about campus life, classes, or resources!")]

    def respond(message, chat_history):
        if not chat_history:
            chat_history = greet()
        bot_reply = chat_with_bot(message)
        chat_history.append((message, bot_reply))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    send.click(respond, [msg, chatbot], [msg, chatbot])

demo.launch(share=True)
